In [68]:
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Iterator
from enum import Enum
import asyncio
from pathlib import Path
import numpy as np
from collections import defaultdict, deque
import mido
import time
import subprocess
import os

# Core Domain Entities

@dataclass
class Note:
    """Represents a musical note with timing information"""
    pitch: int  # MIDI note number (0-127)
    velocity: int  # Note velocity (0-127)
    start_time: float  # Absolute time when note starts
    duration: float  # How long the note plays
    channel: int = 0

@dataclass
class Rest:
    """Represents silence between notes"""
    start_time: float
    duration: float

@dataclass
class Track:
    """A sequence of musical events in a single track"""
    name: str
    notes: List[Note] = field(default_factory=list)
    rests: List[Rest] = field(default_factory=list)
    ticks_per_beat: int = 480
    tempo: int = 500000  # microseconds per beat

@dataclass
class TransitionTable:
    """Markov chain transition probabilities"""
    transitions: Dict[str, Dict[str, float]] = field(default_factory=dict)
    kemeny_constant: Optional[float] = None
    entropy: Optional[float] = None

# MIDI Analysis Classes

class MidiEventExtractor:
    """Extracts musical events from MIDI data"""
    
    def __init__(self):
        self.active_notes: Dict[int, float] = {}  # note -> start_time
        self.last_event_time = 0.0
        
    def extract_track(self, midi_track, ticks_per_beat: int = 480) -> Track:
        """Extract Notes and Rests from a MIDI track"""
        track = Track(name=getattr(midi_track, 'name', 'Untitled'))
        track.ticks_per_beat = ticks_per_beat
        
        current_time = 0.0
        last_note_end = 0.0
        
        for message in midi_track:
            current_time += self._ticks_to_seconds(message.time, ticks_per_beat)
            
            if message.type == 'note_on' and message.velocity > 0:
                # Check for rest before this note
                if current_time > last_note_end and len(track.notes) > 0:
                    rest_duration = current_time - last_note_end
                    track.rests.append(Rest(last_note_end, rest_duration))
                
                self.active_notes[message.note] = current_time
                
            elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
                if message.note in self.active_notes:
                    start_time = self.active_notes.pop(message.note)
                    duration = current_time - start_time
                    
                    note = Note(
                        pitch=message.note,
                        velocity=getattr(message, 'velocity', 64),
                        start_time=start_time,
                        duration=duration,
                        channel=message.channel
                    )
                    track.notes.append(note)
                    last_note_end = max(last_note_end, current_time)
        
        return track
    
    def _ticks_to_seconds(self, ticks: int, ticks_per_beat: int, 
                         tempo: int = 500000) -> float:
        """Convert MIDI ticks to seconds"""
        return (ticks / ticks_per_beat) * (tempo / 1_000_000)

class MarkovChainBuilder:
    """Builds Markov chains from musical sequences"""
    
    def build_note_chain(self, notes: List[Note]) -> TransitionTable:
        """Build transition table for note sequences"""
        transitions = defaultdict(lambda: defaultdict(int))
        
        for i in range(len(notes) - 1):
            current = str(notes[i].pitch)
            next_note = str(notes[i + 1].pitch)
            transitions[current][next_note] += 1
        
        # Convert counts to probabilities
        return self._normalize_transitions(transitions)
    
    def build_duration_chain(self, durations: List[float]) -> TransitionTable:
        """Build transition table for note/rest durations"""
        # Quantize durations to make them discrete
        quantized = [self._quantize_duration(d) for d in durations]
        
        transitions = defaultdict(lambda: defaultdict(int))
        for i in range(len(quantized) - 1):
            current = str(quantized[i])
            next_dur = str(quantized[i + 1])
            transitions[current][next_dur] += 1
            
        return self._normalize_transitions(transitions)
    
    def build_content_chain(self, track: Track) -> TransitionTable:
        """Build chain including both notes and rests"""
        # Merge notes and rests into chronological sequence
        events = []
        for note in track.notes:
            events.append(('note', str(note.pitch), note.start_time))
        for rest in track.rests:
            events.append(('rest', 'rest', rest.start_time))
        
        # Sort by time
        events.sort(key=lambda x: x[2])
        
        transitions = defaultdict(lambda: defaultdict(int))
        for i in range(len(events) - 1):
            current = events[i][1]  # note pitch or 'rest'
            next_event = events[i + 1][1]
            transitions[current][next_event] += 1
            
        return self._normalize_transitions(transitions)
    
    def _quantize_duration(self, duration: float, 
                          quantization: float = 0.1) -> float:
        """Quantize duration to discrete values"""
        return round(duration / quantization) * quantization
    
    def _normalize_transitions(self, transitions: Dict) -> TransitionTable:
        """Convert transition counts to probabilities"""
        table = TransitionTable()
        
        for state, next_states in transitions.items():
            total = sum(next_states.values())
            table.transitions[state] = {
                next_state: count / total 
                for next_state, count in next_states.items()
            }
        
        # Calculate Kemeny constant and entropy
        table.kemeny_constant = self._calculate_kemeny_constant(table)
        table.entropy = self._calculate_entropy(table)
        
        return table
    
    def _calculate_kemeny_constant(self, table: TransitionTable) -> float:
        """Calculate Kemeny constant (simplified version)"""
        # This is a placeholder - full calculation requires matrix operations
        return len(table.transitions)
    
    def _calculate_entropy(self, table: TransitionTable) -> float:
        """Calculate Shannon entropy of transition table"""
        total_entropy = 0.0
        for state, transitions in table.transitions.items():
            state_entropy = 0.0
            for prob in transitions.values():
                if prob > 0:
                    state_entropy -= prob * np.log2(prob)
            total_entropy += state_entropy
        return total_entropy / len(table.transitions) if table.transitions else 0.0

class EntropyAnalyzer:
    """Analyzes entropy in sliding windows"""
    
    def __init__(self, window_size: int = 10):
        self.window_size = window_size
    
    def analyze_track_entropy(self, track: Track) -> List[Tuple[float, float]]:
        """Returns list of (entropy, timestamp) pairs"""
        # Create sequence of events with timestamps
        events = []
        for note in track.notes:
            events.append((str(note.pitch), note.start_time))
        for rest in track.rests:
            events.append(('rest', rest.start_time))
        
        events.sort(key=lambda x: x[1])  # Sort by time
        
        entropies = []
        for i in range(len(events) - self.window_size + 1):
            window_events = [e[0] for e in events[i:i + self.window_size]]
            entropy = self._calculate_window_entropy(window_events)
            timestamp = events[i][1]
            entropies.append((entropy, timestamp))
        
        return entropies
    
    def _calculate_window_entropy(self, events: List[str]) -> float:
        """Calculate entropy for a window of events"""
        from collections import Counter
        counts = Counter(events)
        total = len(events)
        
        entropy = 0.0
        for count in counts.values():
            prob = count / total
            entropy -= prob * np.log2(prob)
        
        return entropy



class MidiComposer:
    """Generates new MIDI compositions from Markov chains"""
    
    def __init__(self):
        self.rng = np.random.default_rng()
    
    def generate_multi_track_composition(self, generator, length: int = 100) -> List[Track]:
        """Generate composition with all learned tracks"""
        compositions = []
        
        for track_name in generator.note_chains.keys():
            if track_name in generator.duration_chains and track_name in generator.content_chains:
                print(f"🎵 Generating track: {track_name}")
                
                track = self.generate_track(
                    generator.note_chains[track_name],
                    generator.duration_chains[track_name], 
                    generator.content_chains[track_name],
                    length
                )
                track.name = f"Generated_{track_name}"
                compositions.append(track)
        
        return compositions
    
    def generate_track(self, note_chain: TransitionTable, 
                      duration_chain: TransitionTable,
                      content_chain: TransitionTable,
                      length: int = 100) -> Track:
        """Generate a single track with proper duration variety"""
        track = Track(name="Generated")
        current_time = 0.0
        
        # Start with most common states
        current_note = self._get_most_common_state(note_chain)
        current_duration = self._get_most_common_state(duration_chain) 
        current_rest_duration = "0.1"  # Default rest
        
        print(f"🎼 Starting composition with note {current_note}, duration {current_duration}")
        
        for i in range(length):
            # Decide if this should be a note or rest based on content chain
            content_choice = self._sample_from_chain(content_chain, current_note)
            
            if content_choice != 'rest' and current_note != 'rest':
                # Generate a note
                try:
                    duration = float(current_duration)
                    note = Note(
                        pitch=int(current_note),
                        velocity=self.rng.integers(60, 100),  # Vary velocity too
                        start_time=current_time,
                        duration=duration
                    )
                    track.notes.append(note)
                    current_time += duration
                    
                    # Get next note and duration
                    current_note = self._sample_from_chain(note_chain, current_note)
                    current_duration = self._sample_from_chain(duration_chain, current_duration)
                    
                except (ValueError, KeyError) as e:
                    print(f"⚠️  Skipping invalid note/duration: {e}")
                    continue
            
            else:
                # Generate a rest
                try:
                    rest_duration = float(current_rest_duration)
                    if rest_duration > 0:
                        rest = Rest(start_time=current_time, duration=rest_duration)
                        track.rests.append(rest)
                        current_time += rest_duration
                    
                    # Get next rest duration and note
                    current_rest_duration = self._sample_from_chain(duration_chain, current_rest_duration)
                    current_note = self._sample_from_chain(content_chain, 'rest')
                    
                except (ValueError, KeyError) as e:
                    print(f"⚠️  Skipping invalid rest: {e}")
                    continue
        
        print(f"✅ Generated track with {len(track.notes)} notes and {len(track.rests)} rests")
        print(f"📊 Duration range: {min([n.duration for n in track.notes]) if track.notes else 0:.2f} - {max([n.duration for n in track.notes]) if track.notes else 0:.2f}")
        
        return track
    
    def _get_most_common_state(self, chain: TransitionTable) -> str:
        """Get the most probable starting state"""
        if not chain.transitions:
            return "60"  # Middle C fallback
        
        # Find state with most outgoing transitions (most connected)
        best_state = max(chain.transitions.keys(), 
                        key=lambda k: len(chain.transitions[k]))
        return best_state
    
    def _sample_from_chain(self, chain: TransitionTable, 
                          current_state: str) -> str:
        """Sample next state from transition probabilities"""
        if current_state not in chain.transitions:
            # Fallback to most common state
            return self._get_most_common_state(chain)
        
        transitions = chain.transitions[current_state]
        if not transitions:
            return current_state
        
        states = list(transitions.keys())
        probabilities = list(transitions.values())
        
        # Normalize probabilities (just in case)
        prob_sum = sum(probabilities)
        if prob_sum > 0:
            probabilities = [p / prob_sum for p in probabilities]
        else:
            # Equal probability fallback
            probabilities = [1.0 / len(states)] * len(states)
        
        return self.rng.choice(states, p=probabilities)




class MidiInterface:
    """Handles MIDI I/O operations"""
    
    def __init__(self):
        self.input_port: Optional[mido.ports.BaseInput] = None
        self.output_port: Optional[mido.ports.BaseOutput] = None
    
    def list_ports(self) -> Tuple[List[str], List[str]]:
        """List available MIDI input and output ports"""
        return mido.get_input_names(), mido.get_output_names()
    
    def open_ports(self, input_name: Optional[str] = None, 
                   output_name: Optional[str] = None):
        """Open MIDI ports for I/O"""
        try:
            if input_name:
                self.input_port = mido.open_input(input_name)
            if output_name:
                self.output_port = mido.open_output(output_name)
        except Exception as e:
            print(f"Failed to open MIDI ports: {e}")
    
    def load_midi_file(self, filepath: Path) -> List[Track]:
        """Load MIDI file and extract tracks"""
        midi_file = mido.MidiFile(filepath)
        extractor = MidiEventExtractor()
        
        tracks = []
        for midi_track in midi_file.tracks:
            track = extractor.extract_track(midi_track, midi_file.ticks_per_beat)
            if track.notes:  # Only include tracks with notes
                tracks.append(track)
        
        return tracks
    
    def save_track_to_midi(self, track: Track, filepath: Path):
        """Save a Track object to MIDI file"""
        midi_file = mido.MidiFile(ticks_per_beat=track.ticks_per_beat)
        midi_track = mido.MidiTrack()
        midi_file.tracks.append(midi_track)
        
        # Convert Track back to MIDI messages
        events = []
        
        # Add note events
        for note in track.notes:
            events.append((note.start_time, 'note_on', note))
            events.append((note.start_time + note.duration, 'note_off', note))
        
        # Sort by time
        events.sort(key=lambda x: x[0])
        
        current_time = 0.0
        for event_time, event_type, note in events:
            delta_time = int((event_time - current_time) * track.ticks_per_beat)
            current_time = event_time
            
            if event_type == 'note_on':
                msg = mido.Message('note_on', channel=note.channel,
                                 note=note.pitch, velocity=note.velocity,
                                 time=delta_time)
            else:
                msg = mido.Message('note_off', channel=note.channel,
                                 note=note.pitch, velocity=0,
                                 time=delta_time)
            
            midi_track.append(msg)
        
        midi_file.save(filepath)





# Main Application Class

class MarkovMidiGenerator:
    """Main application orchestrating all components"""
    
    def __init__(self):
        self.midi_interface = MidiInterface()
        self.chain_builder = MarkovChainBuilder()
        self.entropy_analyzer = EntropyAnalyzer()
        self.composer = MidiComposer()
        
        # Storage for learned patterns
        self.tracks: List[Track] = []
        self.note_chains: Dict[str, TransitionTable] = {}
        self.duration_chains: Dict[str, TransitionTable] = {}
        self.content_chains: Dict[str, TransitionTable] = {}
    
    def analyze_midi_file(self, filepath: Path) -> Dict[str, any]:
        """Analyze a MIDI file and build all Markov chains"""
        self.tracks = self.midi_interface.load_midi_file(filepath)
        
        results = {}
        for i, track in enumerate(self.tracks):
            track_name = f"track_{i}"
            
            # Build chains
            note_chain = self.chain_builder.build_note_chain(track.notes)
            duration_chain = self.chain_builder.build_duration_chain(
                [n.duration for n in track.notes]
            )
            content_chain = self.chain_builder.build_content_chain(track)
            
            # Store chains
            self.note_chains[track_name] = note_chain
            self.duration_chains[track_name] = duration_chain
            self.content_chains[track_name] = content_chain
            
            # Analyze entropy
            entropy_analysis = self.entropy_analyzer.analyze_track_entropy(track)
            
            results[track_name] = {
                'notes': len(track.notes),
                'rests': len(track.rests),
                'kemeny_constant': note_chain.kemeny_constant,
                'entropy': note_chain.entropy,
                'entropy_timeline': entropy_analysis
            }
        
        return results
    
    def generate_composition(self, track_name: str, length: int = 100) -> Track:
        """Generate new composition based on learned patterns"""
        if track_name not in self.note_chains:
            raise ValueError(f"No learned patterns for track: {track_name}")
        
        note_chain = self.note_chains[track_name]
        duration_chain = self.duration_chains[track_name]
        content_chain = self.content_chains[track_name]
        
        # Use content chain for rest patterns
        return self.composer.generate_track(
            note_chain, duration_chain, content_chain, length
        )



    def save_single_track_with_sustaining_instrument(self, track: Track, filepath: Path, instrument_choice: str = "strings"):
        """Save single track with a sustaining instrument that shows note duration clearly"""
        import mido
        
        midi_file = mido.MidiFile(type=0, ticks_per_beat=480)  # Type 0 = single track
        midi_track = mido.MidiTrack()
        midi_file.tracks.append(midi_track)
        
        # Choose sustaining instruments based on preference
        instrument_options = {
            "strings": (40, "Violin"),           # Clear and expressive
            "cello": (42, "Cello"),             # Rich and warm
            "flute": (73, "Flute"),             # Pure and clear
            "clarinet": (71, "Clarinet"),       # Warm woodwind
            "trumpet": (56, "Trumpet"),         # Bright brass
            "french_horn": (60, "French Horn"), # Noble brass
            "organ": (19, "Church Organ"),      # Perfect for your background!
            "synth_pad": (88, "New Age Pad"),   # Modern sustain
        }
        
        if instrument_choice not in instrument_options:
            instrument_choice = "strings"  # Default fallback
        
        instrument_num, instrument_name = instrument_options[instrument_choice]
        
        # Set track name
        track_name = f"{instrument_name}: {track.name}"
        midi_track.append(mido.MetaMessage('track_name', name=track_name, time=0))
        
        # Set instrument
        midi_track.append(mido.Message('program_change', channel=0, program=instrument_num, time=0))
        
        # Add musical settings
        midi_track.append(mido.Message('control_change', channel=0, control=7, value=100, time=0))  # Volume
        midi_track.append(mido.Message('control_change', channel=0, control=91, value=50, time=0))  # Reverb
        
        print(f"🎵 Saving single track with {instrument_name}")
        
        # Convert track to MIDI events
        events = []
        for note in track.notes:
            events.append((note.start_time, 'note_on', note))
            events.append((note.start_time + note.duration, 'note_off', note))
        
        # Sort by time
        events.sort(key=lambda x: x[0])
        
        current_time = 0.0
        for event_time, event_type, note in events:
            delta_ticks = int((event_time - current_time) * 480)
            current_time = event_time
            
            if event_type == 'note_on':
                msg = mido.Message('note_on', channel=0, note=note.pitch, 
                                 velocity=note.velocity, time=delta_ticks)
            else:
                msg = mido.Message('note_off', channel=0, note=note.pitch, 
                                 velocity=0, time=delta_ticks)
            
            midi_track.append(msg)
        
        # End of track
        midi_track.append(mido.MetaMessage('end_of_track', time=0))
        
        midi_file.save(filepath)
        print(f"💾 Saved single track ({instrument_name}) to {filepath}")
    
    # Convenience method for single track generation with better instruments
    def generate_single_track_with_sustaining_instrument(self, track_name: str, length: int = 50, instrument: str = "strings"):
        """Generate and save single track with sustaining instrument"""
        
        # Generate single track
        single_track = self.generate_composition(track_name, length)
        
        # Save with sustaining instrument
        output_path = Path(f"/Users/abraxas3d/organ_donor/data/generated/single_track_{instrument}_{track_name}.mid")
        self.save_single_track_with_sustaining_instrument(single_track, output_path, instrument)
        
        print(f"📊 Generated single track: {len(single_track.notes)} notes, {len(single_track.rests)} rests")
        
        return output_path













    




    

    def generate_multi_track_composition(self, length: int = 100) -> List[Track]:
        """Generate composition with all learned tracks"""
        return self.composer.generate_multi_track_composition(self, length)
    
    def save_multi_track_composition(self, tracks: List[Track], filepath: Path):
        """Save multiple tracks to a single MIDI file"""
        import mido
        
        midi_file = mido.MidiFile(ticks_per_beat=480)
        
        for track in tracks:
            midi_track = mido.MidiTrack()
            midi_track.name = track.name
            midi_file.tracks.append(midi_track)
            
            # Convert Track to MIDI messages
            events = []
            
            # Add note events
            for note in track.notes:
                events.append((note.start_time, 'note_on', note))
                events.append((note.start_time + note.duration, 'note_off', note))
            
            # Sort by time
            events.sort(key=lambda x: x[0])
            
            current_time = 0.0
            for event_time, event_type, note in events:
                delta_time = int((event_time - current_time) * 480)  # Convert to ticks
                current_time = event_time
                
                if event_type == 'note_on':
                    msg = mido.Message('note_on', channel=note.channel,
                                     note=note.pitch, velocity=note.velocity,
                                     time=delta_time)
                else:
                    msg = mido.Message('note_off', channel=note.channel,
                                     note=note.pitch, velocity=0,
                                     time=delta_time)
                
                midi_track.append(msg)
        
        midi_file.save(filepath)
        print(f"💾 Saved {len(tracks)} tracks to {filepath}")



    def save_multi_track_composition_with_sustaining_instruments(self, tracks: List[Track], filepath: Path):
        """Save multiple tracks with instruments that showcase duration and musical patterns"""
        import mido
        
        midi_file = mido.MidiFile(type=1, ticks_per_beat=480)
        
        # Sustaining instruments perfect for hearing note duration
        instrument_families = {
            0: ("Strings", [40, 41, 42, 43]),           # Violin, Viola, Cello, Bass
            1: ("Woodwinds", [73, 74, 71, 68]),        # Flute, Recorder, Clarinet, Oboe
            2: ("Brass", [56, 57, 60, 58]),            # Trumpet, Trombone, French Horn, Tuba
            3: ("Organs", [16, 17, 18, 19]),           # Various organ sounds
            4: ("Synth Pads", [88, 89, 90, 91]),       # Sustained synths
        }
        
        for i, track in enumerate(tracks):
            midi_track = mido.MidiTrack()
            
            # Choose instrument family and specific instrument
            family_idx = i % len(instrument_families)
            family_name, family_instruments = list(instrument_families.values())[family_idx]
            instrument = family_instruments[i % len(family_instruments)]
            
            # Create descriptive track name
            track_name = f"{family_name} {i+1}: {track.name}"
            midi_track.append(mido.MetaMessage('track_name', name=track_name, time=0))
            
            # Set unique channel
            channel = i % 16
            
            # Set instrument
            midi_track.append(mido.Message('program_change', channel=channel, program=instrument, time=0))
            
            # Add musical expression
            midi_track.append(mido.Message('control_change', channel=channel, control=7, value=100, time=0))  # Volume
            midi_track.append(mido.Message('control_change', channel=channel, control=91, value=40, time=0))  # Reverb
            
            print(f"🎺 Track {i+1}: {family_name}, Instrument {instrument}, Channel {channel}")
            
            # Convert notes to MIDI events
            events = []
            for note in track.notes:
                # Add slight velocity variation for musical expression
                musical_velocity = max(40, min(127, note.velocity + np.random.randint(-20, 20)))
                events.append((note.start_time, 'note_on', note, channel, musical_velocity))
                events.append((note.start_time + note.duration, 'note_off', note, channel))
            
            events.sort(key=lambda x: x[0])
            
            current_time = 0.0
            for event_time, event_type, note, ch, *velocity in events:
                delta_ticks = int((event_time - current_time) * 480)
                current_time = event_time
                
                if event_type == 'note_on':
                    vel = velocity[0] if velocity else note.velocity
                    msg = mido.Message('note_on', channel=ch, note=note.pitch, 
                                     velocity=vel, time=delta_ticks)
                else:
                    msg = mido.Message('note_off', channel=ch, note=note.pitch, 
                                     velocity=0, time=delta_ticks)
                
                midi_track.append(msg)
            
            # End of track
            midi_track.append(mido.MetaMessage('end_of_track', time=0))
            midi_file.tracks.append(midi_track)
        
        midi_file.save(filepath)
        print(f"🎼 Saved {len(tracks)} tracks with sustaining instruments to {filepath}")
        print("🎵 These instruments will make note durations and musical patterns much clearer!")

    def save_with_organ_stops(self, tracks: List[Track], filepath: Path):
        """Create composition using different organ stops - perfect for your pipe organ!"""
        import mido
        
        midi_file = mido.MidiFile(type=1, ticks_per_beat=480)
        
        # Pipe organ stops
        organ_stops = [
            (19, "Principal 8'"),        # Church Organ - foundation stop
            (16, "Flute 8'"),           # Drawbar Organ - flute-like 
            (17, "Reed 8'"),            # Percussive Organ - reed stop
            (18, "Mixture"),            # Rock Organ - bright mixture
            (20, "Reed Organ"),         # Reed Organ - different reed
            (88, "String 8'"),          # Pad - string-like stop
            (89, "Bourdon 16'"),        # Warm Pad - soft foundation
        ]
        
        for i, track in enumerate(tracks):
            midi_track = mido.MidiTrack()
            
            # Select organ stop
            instrument_num, stop_name = organ_stops[i % len(organ_stops)]
            
            track_name = f"Organ {stop_name}: {track.name}"
            midi_track.append(mido.MetaMessage('track_name', name=track_name, time=0))
            
            channel = i % 16
            midi_track.append(mido.Message('program_change', channel=channel, program=instrument_num, time=0))
            
            # Organ-specific MIDI settings
            midi_track.append(mido.Message('control_change', channel=channel, control=7, value=110, time=0))  # Volume
            midi_track.append(mido.Message('control_change', channel=channel, control=91, value=60, time=0))  # Reverb
            
            print(f"🏛️  Track {i+1}: {stop_name}, Channel {channel}")
            
            # Add notes
            events = []
            for note in track.notes:
                events.append((note.start_time, 'note_on', note, channel))
                events.append((note.start_time + note.duration, 'note_off', note, channel))
            
            events.sort(key=lambda x: x[0])
            
            current_time = 0.0
            for event_time, event_type, note, ch in events:
                delta_ticks = int((event_time - current_time) * 480)
                current_time = event_time
                
                if event_type == 'note_on':
                    msg = mido.Message('note_on', channel=ch, note=note.pitch, 
                                     velocity=note.velocity, time=delta_ticks)
                else:
                    msg = mido.Message('note_off', channel=ch, note=note.pitch, 
                                     velocity=0, time=delta_ticks)
                
                midi_track.append(msg)
            
            midi_track.append(mido.MetaMessage('end_of_track', time=0))
            midi_file.tracks.append(midi_track)
        
        midi_file.save(filepath)
        print(f"🎼 Saved organ composition with {len(tracks)} stops to {filepath}")

    def generate_and_save_with_better_instruments(self, length: int = 40):
        """Convenience method to generate and save with sustaining instruments"""
        
        # Generate composition
        compositions = self.generate_multi_track_composition(length)
        
        # Save with sustaining instruments
        sustaining_path = Path("data/generated/composition_sustaining.mid")
        sustaining_path.parent.mkdir(exist_ok=True)
        self.save_multi_track_composition_with_sustaining_instruments(compositions, sustaining_path)
        
        # Save with organ stops
        organ_path = Path("data/generated/composition_organ.mid")
        self.save_with_organ_stops(compositions, organ_path)
        
        print(f"\n🎵 Created two versions:")
        print(f"1. Sustaining instruments: {sustaining_path}")
        print(f"2. Organ stops: {organ_path}")
        
        return sustaining_path, organ_path




    def open_in_garageband(midi_path):
        """Open MIDI file in GarageBand (macOS)"""
        try:
            subprocess.run(["open", "-a", "GarageBand", str(midi_path)])
            print(f"🎵 Opening {midi_path.name} in GarageBand...")
        except Exception as e:
            print(f"Error opening GarageBand: {e}")
            # Fallback to default app
            subprocess.run(["open", str(midi_path)])

In [69]:
# Test cell - add this to your notebook
def test_basic_functionality():
    """Quick test of the main classes"""
    
    # Test creating a simple note
    note = Note(pitch=60, velocity=80, start_time=0.0, duration=1.0)
    print(f"✅ Created note: {note}")
    
    # Test creating a track
    track = Track(name="Test Track")
    track.notes.append(note)
    print(f"✅ Created track with {len(track.notes)} notes")
    
    # Test MIDI interface
    midi_interface = MidiInterface()
    inputs, outputs = midi_interface.list_ports()
    print(f"✅ Found {len(outputs)} MIDI output ports")
    
    return True

# Run the test
test_basic_functionality()

✅ Created note: Note(pitch=60, velocity=80, start_time=0.0, duration=1.0, channel=0)
✅ Created track with 1 notes
✅ Found 1 MIDI output ports


True

In [70]:
# Check for connected MIDI devices

midi_interface = MidiInterface()
inputs, outputs = midi_interface.list_ports()
print(f"Input ports: {inputs}")
print(f"Output ports: {outputs}")

Input ports: ['GarageBand Virtual Out']
Output ports: ['GarageBand Virtual In']


In [71]:
# Test with actual MIDI data
from pathlib import Path

# Create a simple test MIDI file or use one you have
generator = MarkovMidiGenerator()

# If you have a MIDI file, try:
results = generator.analyze_midi_file(Path("/Users/abraxas3d/organ_donor/data/midi_files/songs/beet.mid"))
#print(results)

# Generate new composition based on learned patterns - single track
track_name = list(results.keys())[0]  # Use first track's patterns
new_composition = generator.generate_composition(track_name, length=100)

# Save the new composition as MIDI
output_path = Path("/Users/abraxas3d/organ_donor/data/generated/bach_inspired_composition.mid") 
output_path.parent.mkdir(exist_ok=True)
generator.midi_interface.save_track_to_midi(new_composition, output_path)

print(f"🎵 Generated new single-track composition: {output_path}")
print(f"📊 New single-track piece has {len(new_composition.notes)} notes and {len(new_composition.rests)} rests")

🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 30 notes and 70 rests
📊 Duration range: 0.10 - 1.00
🎵 Generated new single-track composition: /Users/abraxas3d/organ_donor/data/generated/bach_inspired_composition.mid
📊 New single-track piece has 30 notes and 70 rests


In [72]:
# Generate multiple tracks
compositions = generator.generate_multi_track_composition(length=100)

print(f"\n🎼 Generated {len(compositions)} tracks:")
for track in compositions:
    note_durations = [n.duration for n in track.notes]
    print(f"  {track.name}: {len(track.notes)} notes, duration variety: {len(set(note_durations))} unique durations")
    if note_durations:
        print(f"    Duration range: {min(note_durations):.3f} - {max(note_durations):.3f}")

# Save multi-track composition
multi_track_output_path = Path("/Users/abraxas3d/organ_donor/data/generated/bach_multi_track.mid")
multi_track_output_path.parent.mkdir(exist_ok=True)
generator.save_multi_track_composition(compositions, multi_track_output_path)


# Open in GarageBand
#open_in_garageband(output_path)

🎵 Generating track: track_0
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 33 notes and 67 rests
📊 Duration range: 0.10 - 0.80
🎵 Generating track: track_1
🎼 Starting composition with note 76, duration 0.30000000000000004
✅ Generated track with 21 notes and 79 rests
📊 Duration range: 0.10 - 2.00
🎵 Generating track: track_2
🎼 Starting composition with note 76, duration 0.5
✅ Generated track with 20 notes and 80 rests
📊 Duration range: 0.10 - 0.50
🎵 Generating track: track_3
🎼 Starting composition with note 57, duration 0.5
✅ Generated track with 16 notes and 84 rests
📊 Duration range: 0.10 - 0.70
🎵 Generating track: track_4
🎼 Starting composition with note 64, duration 0.5
✅ Generated track with 28 notes and 72 rests
📊 Duration range: 0.10 - 5.00
🎵 Generating track: track_5
🎼 Starting composition with note 45, duration 0.1
✅ Generated track with 24 notes and 76 rests
📊 Duration range: 0.10 - 4.50
🎵 Generating track: track_6
🎼 Starting composition

In [73]:
# Generate multiple tracks
compositions = generator.generate_multi_track_composition(length=100)

# Save with sustaining instruments  
sustaining_path = Path("/Users/abraxas3d/organ_donor/data/generated/bach_clear_duration.mid")
generator.save_multi_track_composition_with_sustaining_instruments(compositions, sustaining_path)

# Save with organ stops
organ_path = Path("/Users/abraxas3d/organ_donor/data/generated/bach_organ.mid") 
generator.save_with_organ_stops(compositions, organ_path)

# Open in GarageBand
#open_in_garageband(sustaining_path)

🎵 Generating track: track_0
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 22 notes and 78 rests
📊 Duration range: 0.10 - 0.50
🎵 Generating track: track_1
🎼 Starting composition with note 76, duration 0.30000000000000004
✅ Generated track with 23 notes and 77 rests
📊 Duration range: 0.10 - 0.50
🎵 Generating track: track_2
🎼 Starting composition with note 76, duration 0.5
✅ Generated track with 20 notes and 80 rests
📊 Duration range: 0.10 - 0.60
🎵 Generating track: track_3
🎼 Starting composition with note 57, duration 0.5
✅ Generated track with 18 notes and 82 rests
📊 Duration range: 0.10 - 3.30
🎵 Generating track: track_4
🎼 Starting composition with note 64, duration 0.5
✅ Generated track with 17 notes and 83 rests
📊 Duration range: 0.10 - 0.50
🎵 Generating track: track_5
🎼 Starting composition with note 45, duration 0.1
✅ Generated track with 30 notes and 70 rests
📊 Duration range: 0.10 - 0.50
🎵 Generating track: track_6
🎼 Starting composition

In [74]:
# Get available track names from your analysis
available_tracks = list(generator.note_chains.keys())
print(f"Available tracks: {available_tracks}")

# Generate single tracks with different sustaining instruments
track_name = available_tracks[0]  # Use first track

# Try different instruments to hear note durations clearly:
violin_path = generator.generate_single_track_with_sustaining_instrument(track_name, length=300, instrument="strings")
cello_path = generator.generate_single_track_with_sustaining_instrument(track_name, length=300, instrument="cello") 
flute_path = generator.generate_single_track_with_sustaining_instrument(track_name, length=300, instrument="flute")
organ_path = generator.generate_single_track_with_sustaining_instrument(track_name, length=300, instrument="organ")

# Open one in GarageBand
import subprocess
subprocess.run(["open", "-a", "GarageBand", str(cello_path)])

Available tracks: ['track_0', 'track_1', 'track_2', 'track_3', 'track_4', 'track_5', 'track_6', 'track_7', 'track_8', 'track_9', 'track_10', 'track_11']
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 68 notes and 232 rests
📊 Duration range: 0.10 - 2.20
🎵 Saving single track with Violin
💾 Saved single track (Violin) to /Users/abraxas3d/organ_donor/data/generated/single_track_strings_track_0.mid
📊 Generated single track: 68 notes, 232 rests
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 68 notes and 232 rests
📊 Duration range: 0.10 - 0.80
🎵 Saving single track with Cello
💾 Saved single track (Cello) to /Users/abraxas3d/organ_donor/data/generated/single_track_cello_track_0.mid
📊 Generated single track: 68 notes, 232 rests
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 81 notes and 219 rests
📊 Duration range: 0.10 - 1.00
🎵 Saving single track with Flute
💾 Saved sing

CompletedProcess(args=['open', '-a', 'GarageBand', '/Users/abraxas3d/organ_donor/data/generated/single_track_cello_track_0.mid'], returncode=0)

In [75]:
# First, let's check what's actually being generated
print("🔍 Debugging single track generation:")
print(f"Track name being used: {track_name}")

# Generate one track and examine it
single_track = generator.generate_composition(track_name, length=30)
print(f"Generated track has: {len(single_track.notes)} notes, {len(single_track.rests)} rests")

# Check if the save method exists
if hasattr(generator, 'save_single_track_with_sustaining_instrument'):
    print("✅ Single track save method exists")
else:
    print("❌ Single track save method missing - you need to add it to your class")

# Test with a simple save to see track structure
test_path = Path("/Users/abraxas3d/organ_donor/data/generated/debug_single_track.mid")
generator.save_single_track_with_sustaining_instrument(single_track, test_path, "cello")

# Let's examine the MIDI file structure
import mido
debug_file = mido.MidiFile(test_path)
print(f"\n📊 MIDI file analysis:")
print(f"Number of tracks in file: {len(debug_file.tracks)}")
print(f"MIDI file type: {debug_file.type}")

for i, track in enumerate(debug_file.tracks):
    note_count = sum(1 for msg in track if msg.type == 'note_on' and msg.velocity > 0)
    print(f"Track {i}: {note_count} note_on messages")

🔍 Debugging single track generation:
Track name being used: track_0
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 9 notes and 21 rests
📊 Duration range: 0.10 - 0.50
Generated track has: 9 notes, 21 rests
✅ Single track save method exists
🎵 Saving single track with Cello
💾 Saved single track (Cello) to /Users/abraxas3d/organ_donor/data/generated/debug_single_track.mid

📊 MIDI file analysis:
Number of tracks in file: 1
MIDI file type: 0
Track 0: 9 note_on messages


In [76]:
# Create a fresh single-track test file with a unique name
fresh_path = Path("/Users/abraxas3d/organ_donor/data/generated/SINGLE_TRACK_TEST.mid")
generator.save_single_track_with_sustaining_instrument(single_track, fresh_path, "cello")

# Verify it's single track
import mido
test_file = mido.MidiFile(fresh_path)
print(f"✅ Confirmed: {len(test_file.tracks)} track(s) in {fresh_path.name}")

# Close any open GarageBand files first, then open this specific file
import subprocess
subprocess.run(["open", "-a", "GarageBand", str(fresh_path)])

🎵 Saving single track with Cello
💾 Saved single track (Cello) to /Users/abraxas3d/organ_donor/data/generated/SINGLE_TRACK_TEST.mid
✅ Confirmed: 1 track(s) in SINGLE_TRACK_TEST.mid


CompletedProcess(args=['open', '-a', 'GarageBand', '/Users/abraxas3d/organ_donor/data/generated/SINGLE_TRACK_TEST.mid'], returncode=0)

In [77]:
# Debug why no rests are being generated
print("🔍 Debugging rest generation:")

# Check what's in the content chain (should have both notes and 'rest')
track_name = available_tracks[0]
content_chain = generator.content_chains[track_name]

print(f"Content chain states: {list(content_chain.transitions.keys())}")
print(f"Content chain has 'rest'? {'rest' in content_chain.transitions}")

# Check some transition probabilities
for state, transitions in list(content_chain.transitions.items())[:3]:
    print(f"From '{state}': {transitions}")

# Also check the original track analysis
print(f"\nOriginal analysis showed: {results[track_name]['rests']} rests")

🔍 Debugging rest generation:
Content chain states: ['80', 'rest', '81', '88', '76', '90', '78', '91', '79', '83', '92', '93', '86', '85', '87', '89', '84', '74', '72']
Content chain has 'rest'? True
From '80': {'rest': 0.6, '92': 0.2, '83': 0.06666666666666667, '88': 0.13333333333333333}
From 'rest': {'rest': 0.3002481389578164, '81': 0.09925558312655088, '88': 0.1315136476426799, '76': 0.03970223325062035, '90': 0.01240694789081886, '91': 0.009925558312655087, '79': 0.017369727047146403, '83': 0.08684863523573201, '78': 0.019851116625310174, '80': 0.02729528535980149, '93': 0.004962779156327543, '92': 0.0024813895781637717, '85': 0.022332506203473945, '89': 0.03722084367245657, '84': 0.07692307692307693, '86': 0.09181141439205956, '87': 0.01240694789081886, '74': 0.004962779156327543, '72': 0.0024813895781637717}
From '81': {'88': 0.021739130434782608, 'rest': 0.782608695652174, '93': 0.08695652173913043, '83': 0.06521739130434782, '85': 0.021739130434782608, '72': 0.02173913043478260

In [78]:
# Let's manually trace through the MIDI file to see where rests should be detected
import mido

bach_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/bach.mid")

# Analyze track 0 step by step
track = bach_file.tracks[0]
print("🔍 Manual rest detection analysis:")

current_time = 0.0
active_notes = {}  # note -> start_time
last_note_end = 0.0
rest_count = 0

print("First 20 MIDI messages:")
for i, message in enumerate(track):
    current_time += message.time * (500000 / 1000000) / 480  # Convert ticks to seconds roughly
    
    if message.type == 'note_on' and message.velocity > 0:
        # Note starts
        print(f"{i}: Note ON  {message.note} at time {current_time:.3f}")
        
        # Check if there's a gap since last note ended (= rest)
        if current_time > last_note_end and last_note_end > 0:
            rest_duration = current_time - last_note_end
            print(f"   🎵 REST DETECTED: {rest_duration:.3f} seconds")
            rest_count += 1
        
        active_notes[message.note] = current_time
        
    elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
        # Note ends
        if message.note in active_notes:
            start_time = active_notes.pop(message.note)
            duration = current_time - start_time
            print(f"{i}: Note OFF {message.note} at time {current_time:.3f}, duration {duration:.3f}")
            last_note_end = max(last_note_end, current_time)
    
    if i >= 20:  # Just analyze first 20 messages
        break

print(f"\n📊 Manual analysis found {rest_count} rests in first 20 messages")
print(f"Our algorithm found: {results['track_0']['rests']} rests in entire track")

🔍 Manual rest detection analysis:
First 20 MIDI messages:

📊 Manual analysis found 0 rests in first 20 messages
Our algorithm found: 403 rests in entire track


In [79]:
# Let's see what types of messages are in the MIDI file
import mido

bach_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/bach.mid")
track = bach_file.tracks[0]

print("🔍 All message types in first 30 messages:")
current_time = 0
for i, message in enumerate(track):
    current_time += message.time
    print(f"{i:2d}: {message.type:15} | time:{message.time:4d} | cumulative:{current_time:6d} | {message}")
    
    if i >= 30:
        break

print(f"\n📊 Message type summary:")
message_types = {}
note_on_count = 0
note_off_count = 0

for message in track:
    msg_type = message.type
    message_types[msg_type] = message_types.get(msg_type, 0) + 1
    
    if message.type == 'note_on' and message.velocity > 0:
        note_on_count += 1
    elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
        note_off_count += 1

for msg_type, count in sorted(message_types.items()):
    print(f"  {msg_type}: {count}")

print(f"\nActual note events:")
print(f"  Note ONs: {note_on_count}")
print(f"  Note OFFs: {note_off_count}")

🔍 All message types in first 30 messages:
 0: track_name      | time:   0 | cumulative:     0 | MetaMessage('track_name', name='untitled', time=0)
 1: smpte_offset    | time:   0 | cumulative:     0 | MetaMessage('smpte_offset', frame_rate=30, hours=0, minutes=0, seconds=3, frames=0, sub_frames=0, time=0)
 2: time_signature  | time:   0 | cumulative:     0 | MetaMessage('time_signature', numerator=12, denominator=8, clocks_per_click=12, notated_32nd_notes_per_beat=8, time=0)
 3: key_signature   | time:   0 | cumulative:     0 | MetaMessage('key_signature', key='F', time=0)
 4: set_tempo       | time:   0 | cumulative:     0 | MetaMessage('set_tempo', tempo=240000, time=0)
 5: set_tempo       | time:2640 | cumulative:  2640 | MetaMessage('set_tempo', tempo=444444, time=2640)
 6: marker          | time: 240 | cumulative:  2880 | MetaMessage('marker', text='A', time=240)
 7: marker          | time:57600 | cumulative: 60480 | MetaMessage('marker', text="A'", time=57600)
 8: marker         

In [80]:
# Check all tracks in the Bach file
bach_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/bach.mid")
print(f"🎼 Bach file has {len(bach_file.tracks)} tracks total")

for track_num, track in enumerate(bach_file.tracks):
    note_on_count = 0
    note_off_count = 0
    message_count = len(track)
    
    for message in track:
        if message.type == 'note_on' and message.velocity > 0:
            note_on_count += 1
        elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
            note_off_count += 1
    
    print(f"Track {track_num}: {message_count} messages, {note_on_count} note_ons, {note_off_count} note_offs")

# Let's analyze a track that actually has notes
for track_num, track in enumerate(bach_file.tracks):
    if any(msg.type == 'note_on' and msg.velocity > 0 for msg in track):
        print(f"\n🎵 Analyzing Track {track_num} (has notes):")
        
        # Re-run our analysis on a track with actual notes
        extractor = MidiEventExtractor()
        analyzed_track = extractor.extract_track(track, bach_file.ticks_per_beat)
        
        print(f"Found: {len(analyzed_track.notes)} notes, {len(analyzed_track.rests)} rests")
        
        # Show first few notes and their timing
        for i, note in enumerate(analyzed_track.notes[:5]):
            print(f"  Note {i}: pitch {note.pitch}, start {note.start_time:.3f}, duration {note.duration:.3f}")
        
        for i, rest in enumerate(analyzed_track.rests[:5]):
            print(f"  Rest {i}: start {rest.start_time:.3f}, duration {rest.duration:.3f}")
        
        break  # Just analyze the first track with notes

🎼 Bach file has 17 tracks total
Track 0: 12 messages, 0 note_ons, 0 note_offs
Track 1: 3638 messages, 1816 note_ons, 1816 note_offs
Track 2: 3 messages, 0 note_ons, 0 note_offs
Track 3: 3 messages, 0 note_ons, 0 note_offs
Track 4: 3 messages, 0 note_ons, 0 note_offs
Track 5: 3 messages, 0 note_ons, 0 note_offs
Track 6: 3 messages, 0 note_ons, 0 note_offs
Track 7: 3 messages, 0 note_ons, 0 note_offs
Track 8: 3 messages, 0 note_ons, 0 note_offs
Track 9: 3 messages, 0 note_ons, 0 note_offs
Track 10: 3 messages, 0 note_ons, 0 note_offs
Track 11: 3 messages, 0 note_ons, 0 note_offs
Track 12: 3 messages, 0 note_ons, 0 note_offs
Track 13: 3 messages, 0 note_ons, 0 note_offs
Track 14: 3 messages, 0 note_ons, 0 note_offs
Track 15: 3 messages, 0 note_ons, 0 note_offs
Track 16: 3 messages, 0 note_ons, 0 note_offs

🎵 Analyzing Track 1 (has notes):
Found: 1816 notes, 0 rests
  Note 0: pitch 69, start 2.750, duration 0.250
  Note 1: pitch 62, start 3.000, duration 0.250
  Note 2: pitch 65, start 3.2

In [81]:
# Check if ALL notes are perfectly connected
track1_notes = analyzed_track.notes

gaps = []
overlaps = []
perfect_connections = 0

for i in range(len(track1_notes) - 1):
    current_end = track1_notes[i].start_time + track1_notes[i].duration
    next_start = track1_notes[i + 1].start_time
    
    gap = next_start - current_end
    
    if abs(gap) < 0.001:  # Perfectly connected (within 1ms)
        perfect_connections += 1
    elif gap > 0:  # Actual gap (rest)
        gaps.append(gap)
    else:  # Overlap
        overlaps.append(abs(gap))

print(f"📊 Note connection analysis:")
print(f"Perfect connections: {perfect_connections}")
print(f"Actual gaps (rests): {len(gaps)}")
print(f"Overlaps: {len(overlaps)}")

#if gaps:
#    print(f"Gap sizes: min={min(gaps):.3f}, max={max(gaps):.3f}, avg={sum(gaps)/len(gaps):.3f}")

📊 Note connection analysis:
Perfect connections: 1815
Actual gaps (rests): 0
Overlaps: 0


In [82]:
# Analyze the Beethoven file
beet_results = generator.analyze_midi_file(Path("/Users/abraxas3d/organ_donor/data/midi_files/songs/beet.mid"))
print("🎼 Beethoven analysis results:")
print(beet_results)

# Check track structure
import mido
beet_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/beet.mid")
print(f"\n📊 Beethoven file has {len(beet_file.tracks)} tracks")

for track_num, track in enumerate(beet_file.tracks):
    note_on_count = sum(1 for msg in track if msg.type == 'note_on' and msg.velocity > 0)
    note_off_count = sum(1 for msg in track if msg.type in ['note_off', 'note_on'] and msg.velocity == 0)
    print(f"Track {track_num}: {note_on_count} note_ons, {note_off_count} note_offs")

# Check content chains for 'rest' states
print(f"\n🔍 Checking for rests in Beethoven analysis:")
for track_name, chain in generator.content_chains.items():
    has_rest = 'rest' in chain.transitions
    states = list(chain.transitions.keys())
    print(f"{track_name}: has 'rest'? {has_rest}, total states: {len(states)}")
    if has_rest:
        print(f"  Rest transitions: {chain.transitions['rest']}")

🎼 Beethoven analysis results:
{'track_0': {'notes': 404, 'rests': 403, 'kemeny_constant': 18, 'entropy': 1.8837179379908369, 'entropy_timeline': [(1.9609640474436814, 73.5), (1.7609640474436812, 74.003125), (1.7609640474436812, 74.003125), (1.7609640474436812, 74.003125), (1.8464393446710154, 74.49895833333333), (1.5709505944546684, 74.99791666666667), (1.4854752972273344, 74.99791666666667), (1.3709505944546687, 75.42708333333333), (1.4854752972273344, 75.42708333333333), (1.5709505944546684, 75.49479166666666), (1.4854752972273344, 75.49583333333332), (1.3709505944546687, 75.584375), (1.4854752972273344, 75.584375), (1.5709505944546684, 75.75), (1.4854752972273344, 75.75104166666667), (1.3709505944546687, 75.84166666666667), (1.4854752972273344, 75.84166666666667), (1.5709505944546684, 75.99791666666667), (1.4854752972273344, 76.00104166666667), (1.3709505944546687, 76.41562499999999), (1.4854752972273344, 76.41562499999999), (1.5709505944546684, 76.49791666666665), (1.48547529722733

In [83]:
# Check for 'rest' states in the Beethoven content chains
print("🔍 Checking Beethoven content chains for rests:")
for track_name, chain in generator.content_chains.items():
    has_rest = 'rest' in chain.transitions
    total_states = len(chain.transitions)
    print(f"{track_name}: has 'rest'? {has_rest}, total states: {total_states}")
    
    if has_rest:
        rest_transitions = chain.transitions['rest']
        print(f"  From 'rest' goes to: {list(rest_transitions.keys())[:5]}...")  # Show first 5

🔍 Checking Beethoven content chains for rests:
track_0: has 'rest'? True, total states: 19
  From 'rest' goes to: ['rest', '81', '88', '76', '90']...
track_1: has 'rest'? True, total states: 21
  From 'rest' goes to: ['rest', '79', '75', '78', '81']...
track_2: has 'rest'? True, total states: 22
  From 'rest' goes to: ['rest', '64', '76', '78', '67']...
track_3: has 'rest'? True, total states: 21
  From 'rest' goes to: ['rest', '60', '66', '62', '64']...
track_4: has 'rest'? True, total states: 11
  From 'rest' goes to: ['rest', '52', '64', '66', '67']...
track_5: has 'rest'? True, total states: 5
  From 'rest' goes to: ['64', 'rest', '57', '45', '66']...
track_6: has 'rest'? True, total states: 4
  From 'rest' goes to: ['52', '45', '51']...
track_7: has 'rest'? True, total states: 30
  From 'rest' goes to: ['76', '78', '79', '81', '83']...
track_8: has 'rest'? True, total states: 28
  From 'rest' goes to: ['64', '66', '67', '69', '71']...
track_9: has 'rest'? True, total states: 26
  

In [84]:
# Generate composition from Beethoven with rests
available_tracks = list(generator.note_chains.keys())
print(f"Available Beethoven tracks: {available_tracks}")

# Use track_0 (which has lots of rests)
beethoven_track = generator.generate_composition('track_0', length=300)
print(f"Generated Beethoven-style: {len(beethoven_track.notes)} notes, {len(beethoven_track.rests)} rests")

# Save with sustaining instrument
beet_path = Path("/Users/abraxas3d/organ_donor/data/generated/beethoven_with_rests.mid")
generator.save_single_track_with_sustaining_instrument(beethoven_track, beet_path, "cello")

print(f"🎵 Saved Beethoven-inspired composition with natural rests!")
print(f"This should have the breathing patterns of the original!")

# Open in GarageBand  
import subprocess
subprocess.run(["open", "-a", "GarageBand", str(beet_path)])

Available Beethoven tracks: ['track_0', 'track_1', 'track_2', 'track_3', 'track_4', 'track_5', 'track_6', 'track_7', 'track_8', 'track_9', 'track_10', 'track_11']
🎼 Starting composition with note 88, duration 0.30000000000000004
✅ Generated track with 67 notes and 233 rests
📊 Duration range: 0.10 - 1.00
Generated Beethoven-style: 67 notes, 233 rests
🎵 Saving single track with Cello
💾 Saved single track (Cello) to /Users/abraxas3d/organ_donor/data/generated/beethoven_with_rests.mid
🎵 Saved Beethoven-inspired composition with natural rests!
This should have the breathing patterns of the original!


CompletedProcess(args=['open', '-a', 'GarageBand', '/Users/abraxas3d/organ_donor/data/generated/beethoven_with_rests.mid'], returncode=0)